## Homework 02 - regression

In [1]:
import pandas as pd
import numpy as np

### Dataset

In this homework, we will use the California Housing Prices from [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices).

Here's a wget-able [link](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv):

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
```

The goal of this homework is to create a regression model for predicting housing prices (column `'median_house_value'`).

### EDA

* Load the data.
* Look at the `median_house_value` variable. Does it have a long tail? 

In [2]:
df = pd.read_csv("housing.csv")
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


### Preparing the dataset 

For this homework, we only want to use a subset of data. 

First, keep only the records where `ocean_proximity` is either `'<1H OCEAN'` or `'INLAND'`

Next, use only the following columns:

* `'latitude'`,
* `'longitude'`,
* `'housing_median_age'`,
* `'total_rooms'`,
* `'total_bedrooms'`,
* `'population'`,
* `'households'`,
* `'median_income'`,
* `'median_house_value'`

In [3]:
# 1.
df = df[df["ocean_proximity"].isin(["<1H OCEAN", "INLAND"])]
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
701,-121.97,37.64,32.0,1283.0,194.0,485.0,171.0,6.0574,431000.0,<1H OCEAN
830,-121.99,37.61,9.0,3666.0,711.0,2341.0,703.0,4.6458,217000.0,<1H OCEAN
859,-121.97,37.57,21.0,4342.0,783.0,2172.0,789.0,4.6146,247600.0,<1H OCEAN
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,283500.0,<1H OCEAN
861,-121.98,37.58,20.0,4126.0,1031.0,2079.0,975.0,3.6832,216900.0,<1H OCEAN
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [4]:
# 2.
columns = ["latitude", "longitude", "housing_median_age", "total_rooms", "total_bedrooms", 
           "population", "households", "median_income", "median_house_value"]
df = df[columns]
df

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
701,37.64,-121.97,32.0,1283.0,194.0,485.0,171.0,6.0574,431000.0
830,37.61,-121.99,9.0,3666.0,711.0,2341.0,703.0,4.6458,217000.0
859,37.57,-121.97,21.0,4342.0,783.0,2172.0,789.0,4.6146,247600.0
860,37.58,-121.96,15.0,3575.0,597.0,1777.0,559.0,5.7192,283500.0
861,37.58,-121.98,20.0,4126.0,1031.0,2079.0,975.0,3.6832,216900.0
...,...,...,...,...,...,...,...,...,...
20635,39.48,-121.09,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
20636,39.49,-121.21,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
20637,39.43,-121.22,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
20638,39.43,-121.32,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


### Question 1

There's one feature with missing values. What is it?

* `total_rooms`
* `total_bedrooms`
* `population`
* `households`

In [5]:
df.isna().any()

latitude              False
longitude             False
housing_median_age    False
total_rooms           False
total_bedrooms         True
population            False
households            False
median_income         False
median_house_value    False
dtype: bool

### Question 2

What's the median (50% percentile) for variable `'population'`?

- 995
- 1095
- 1195
- 1295

In [6]:
df.describe()

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,15687.000000,15687.000000,15687.000000,15687.000000,15530.000000,15687.000000,15687.000000,15687.000000,15687.000000
mean,35.467307,-119.217442,27.188118,2665.677312,541.244688,1466.317205,500.916491,3.804019,191943.020017
std,2.066387,1.780380,12.057128,2257.672743,435.650018,1180.389908,392.759399,1.857158,108801.077620
min,32.610000,-124.140000,1.000000,2.000000,2.000000,3.000000,2.000000,0.499900,14999.000000
25%,33.940000,-121.070000,17.000000,1441.000000,295.000000,802.000000,278.000000,2.520800,111300.000000
50%,34.160000,-118.370000,27.000000,2118.000000,432.000000,1195.000000,406.000000,3.468800,166900.000000
75%,37.340000,-117.990000,36.000000,3172.000000,645.000000,1777.000000,602.000000,4.686200,241100.000000
max,41.950000,-114.310000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


### Prepare and split the dataset

* Shuffle the dataset (the filtered one you created above), use seed `42`.
* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Apply the log transformation to the `median_house_value` variable using the `np.log1p()` function.

In [7]:
# shuffle the dataset
n = len(df)
idx = np.arange(n)
np.random.seed(42)
np.random.shuffle(idx)
df_shuffled = df.iloc[idx]

In [8]:
# split the dara
n_val = int(0.2 * n)
n_test = int(0.2 * n)
n_train = n - (n_val + n_test)

x_train = df_shuffled.iloc[:n_train].copy()
x_val = df_shuffled.iloc[n_train:n_train+n_val].copy()
x_test = df_shuffled.iloc[n_train+n_val:].copy()

y_train = np.log1p(x_train.median_house_value.values)
y_val = np.log1p(x_val.median_house_value.values)
y_test = np.log1p(x_test.median_house_value.values)

x_train.drop("median_house_value", axis=1, inplace=True)
x_val.drop("median_house_value", axis=1, inplace=True)
x_test.drop("median_house_value", axis=1, inplace=True)

### Question 3

* We need to deal with missing values for the column from Q1.
* We have two options: fill it with 0 or with the mean of this variable.
* Try both options. For each, train a linear regression model without regularization using the code from the lessons.
* For computing the mean, use the training only!
* Use the validation dataset to evaluate the models and compare the RMSE of each option.
* Round the RMSE scores to 2 decimal digits using `round(score, 2)`
* Which option gives better RMSE?

Options:

- With 0
- With mean
- Both are equally good

In [9]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [10]:
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [11]:
# with 0
x_zero_train = x_train.fillna(0)
w_0_zero, w_zero = train_linear_regression(x_zero_train, y_train)

# val
x_zero_val = x_val.fillna(0)
y_zero_pred_val = w_0_zero + x_zero_val.dot(w_zero)

# rmse
np.round(rmse(y_val, y_zero_pred_val), 2)

0.34

In [12]:
# with mean
mean = x_train.total_bedrooms.mean()

x_mean_train = x_train.fillna(mean)
w_0_mean, w_mean = train_linear_regression(x_mean_train, y_train)

# val
x_mean_val = x_val.fillna(mean)
y_mean_pred_val = w_0_mean + x_mean_val.dot(w_mean)

# rmse
np.round(rmse(y_val, y_mean_pred_val), 2)

0.34

### Question 4

* Now let's train a regularized linear regression.
* For this question, fill the NAs with 0. 
* Try different values of `r` from this list: `[0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]`.
* Use RMSE to evaluate the model on the validation dataset.
* Round the RMSE scores to 2 decimal digits.
* Which `r` gives the best RMSE?

If there are multiple options, select the smallest `r`.

Options:

- 0
- 0.000001
- 0.001
- 0.0001

In [13]:
def train_linear_regression_reg(X, y, r=0.0):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [14]:
for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
    w_0, w = train_linear_regression_reg(x_zero_train, y_train, r=r)
    y_zero_reg_val = w_0 + x_zero_val.dot(w)
    rmse_val = np.round(rmse(y_val, y_zero_reg_val), 2)
    print(r, w_0, rmse_val)

0 -9.763249477929213 0.34
1e-06 -9.76322883158197 0.34
0.0001 -9.761185235939122 0.34
0.001 -9.74264624988528 0.34
0.01 -9.561056193156471 0.34
0.1 -8.058889769818062 0.34
1 -3.1331542785822872 0.34
5 -0.841086797533389 0.35
10 -0.4381172315908744 0.35


### Question 5 

* We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
* Try different seed values: `[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]`.
* For each seed, do the train/validation/test split with 60%/20%/20% distribution.
* Fill the missing values with 0 and train a model without regularization.
* For each seed, evaluate the model on the validation dataset and collect the RMSE scores. 
* What's the standard deviation of all the scores? To compute the standard deviation, use `np.std`.
* Round the result to 3 decimal digits (`round(std, 3)`)

What's the value of std?

- 0.5
- 0.05
- 0.005
- 0.0005

> Note: Standard deviation shows how different the values are.
> If it's low, then all values are approximately the same.
> If it's high, the values are different. 
> If standard deviation of scores is low, then our model is *stable*.

In [15]:
rmse_list = []

for r in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    # shuffle the dataset
    idx = np.arange(n)
    np.random.seed(r)
    np.random.shuffle(idx)
    df_shuffled = df.iloc[idx]
    
    # split the dara
    x_train = df_shuffled.iloc[:n_train].copy()
    x_val = df_shuffled.iloc[n_train:n_train+n_val].copy()
    x_test = df_shuffled.iloc[n_train+n_val:].copy()

    y_train = np.log1p(x_train.median_house_value.values)
    y_val = np.log1p(x_val.median_house_value.values)
    y_test = np.log1p(x_test.median_house_value.values)

    x_train.drop("median_house_value", axis=1, inplace=True)
    x_val.drop("median_house_value", axis=1, inplace=True)
    x_test.drop("median_house_value", axis=1, inplace=True)
    
    # fill missing values with 0
    x_zero_train = x_train.fillna(0)
    w_0_zero, w_zero = train_linear_regression(x_zero_train, y_train)

    # val
    x_zero_val = x_val.fillna(0)
    y_zero_pred_val = w_0_zero + x_zero_val.dot(w_zero)

    # rmse
    rmse_val = np.round(rmse(y_val, y_zero_pred_val), 2)
    rmse_list.append(rmse_val)

In [16]:
rmse_list

[0.34, 0.34, 0.34, 0.33, 0.34, 0.34, 0.34, 0.35, 0.35, 0.33]

In [17]:
np.round(np.std(rmse_list),3)

0.006

### Question 6

* Split the dataset like previously, use seed 9.
* Combine train and validation datasets.
* Fill the missing values with 0 and train a model with `r=0.001`. 
* What's the RMSE on the test dataset?

Options:

- 0.13
- 0.23
- 0.33
- 0.43

In [18]:
# shuffle the dataset
idx = np.arange(n)
np.random.seed(9)
np.random.shuffle(idx)
df_shuffled = df.iloc[idx]

# split the dara
x_train = df_shuffled.iloc[:n_train].copy()
x_val = df_shuffled.iloc[n_train:n_train+n_val].copy()
x_test = df_shuffled.iloc[n_train+n_val:].copy()

x_train_val = pd.concat([x_train, x_val])

y_train_val = np.log1p(x_train_val.median_house_value.values)
y_test = np.log1p(x_test.median_house_value.values)

x_train_val.drop("median_house_value", axis=1, inplace=True)
x_test.drop("median_house_value", axis=1, inplace=True)

# fill missing values with 0
x_zero_train_val = x_train_val.fillna(0)
w_0_zero, w_zero = train_linear_regression_reg(x_zero_train_val, y_train_val, r=0.001)

# test
x_zero_test = x_test.fillna(0)
y_zero_pred_test = w_0_zero + x_zero_test.dot(w_zero)

# rmse
np.round(rmse(y_test, y_zero_pred_test), 2)

0.33